In [1]:
%pip install pgmpy

from pgmpy.base import *
from pgmpy.models import DynamicBayesianNetwork
from pgmpy.inference import DBNInference, Inference
from pgmpy.factors.discrete import TabularCPD
import matplotlib.pyplot as plt
import networkx as nx
import math
import numpy as np
import pandas as pd
import pprint

Note: you may need to restart the kernel to use updated packages.


In [2]:
#np.warnings.filterwarnings('ignore')
#np.seterr(invalid='ignore')

###GBXML in graph
DBN = DynamicBayesianNetwork()

# 엑셀에서 모든 노드 읽어서 추가 및 DAG 작성
node_data = pd.read_excel('nodedata/dbn_node_prob_1381.xlsx',
                          sheet_name = 'Nodes')
node_data

,node name
0,24
1,29
2,35
3,36
4,37
5,47
6,52
7,53
8,54
9,55


In [3]:
node_data
nodes = node_data['node name']
for node in nodes:
    DBN.add_node(node)
    DBN.add_edges_from([((node, 0), (node, 1))])
print(DBN)

DynamicBayesianNetwork with 32 nodes and 16 edges


In [4]:
DAG_data = pd.read_excel('nodedata/dbn_node_prob_1381.xlsx',
                         sheet_name = 'DAG_prob')
initial_nodes = DAG_data['Initial']
final_nodes = DAG_data['Final']
print(initial_nodes.tolist())
print(final_nodes.tolist())

if len(initial_nodes) != len(final_nodes):
    raise ValueError('Nodes Length Error')

[36, 52, 53, 53, 54, 54, 54, 54, 55, 55, 56, 56, 57, 60, 60, 61, 64]
[54, 60, 52, 61, 24, 35, 53, 55, 37, 56, 57, 64, 65, 29, 47, 60, 65]


In [5]:
for num in range(len(initial_nodes)):
    ininode = initial_nodes[num]
    finnode = final_nodes[num]
    DBN.add_edges_from([((ininode, 0), (finnode, 0))])

In [6]:
dNodes = DBN.nodes
dEdges = DBN.edges
#그래프로부터 edge 목록 읽어오기
edgeList = []
for ed in dEdges:
    changeToList = list(ed)
    edgeList.append(changeToList)
                    
print(edgeList)

[[<DynamicNode(24, 0) at 0x2b29413abb0>, <DynamicNode(24, 1) at 0x2b29413f760>], [<DynamicNode(29, 0) at 0x2b29413f1c0>, <DynamicNode(29, 1) at 0x2b29413f2e0>], [<DynamicNode(35, 0) at 0x2b29413faf0>, <DynamicNode(35, 1) at 0x2b29413f430>], [<DynamicNode(36, 0) at 0x2b29413f250>, <DynamicNode(36, 1) at 0x2b29413f880>], [<DynamicNode(36, 0) at 0x2b29413f250>, <DynamicNode(54, 0) at 0x2b286716070>], [<DynamicNode(36, 1) at 0x2b29413f880>, <DynamicNode(54, 1) at 0x2b2867165e0>], [<DynamicNode(37, 0) at 0x2b29413f550>, <DynamicNode(37, 1) at 0x2b29413f3a0>], [<DynamicNode(47, 0) at 0x2b29413fa60>, <DynamicNode(47, 1) at 0x2b29413fbb0>], [<DynamicNode(52, 0) at 0x2b29413f070>, <DynamicNode(52, 1) at 0x2b29413f8b0>], [<DynamicNode(52, 0) at 0x2b29413f070>, <DynamicNode(60, 0) at 0x2b2867165b0>], [<DynamicNode(52, 1) at 0x2b29413f8b0>, <DynamicNode(60, 1) at 0x2b29414dac0>], [<DynamicNode(53, 0) at 0x2b29413fb50>, <DynamicNode(53, 1) at 0x2b29413f6a0>], [<DynamicNode(53, 0) at 0x2b29413fb50>,

In [7]:
# 필요한 conditional probability distribution 목록을 나열
totalEdge = edgeList
evidenceList, variableList = [], []
for nodes in dNodes:
    for edges in totalEdge:
        if edges[1] == nodes:
            variableList.append(edges[1])
            evidenceList.append(edges[0])
        else:
            pass

In [8]:
variableSet = []
for x in variableList:
    if x not in variableSet:
        variableSet.append(x)
#variableSet = l#variableList 중복제거한 List
print(variableSet)
lastVariableList, lastEvidenceList, tabCPDList = [], [], []

[<DynamicNode(24, 0) at 0x2b294225340>, <DynamicNode(24, 1) at 0x2b29413f760>, <DynamicNode(29, 0) at 0x2b294259be0>, <DynamicNode(29, 1) at 0x2b29413f2e0>, <DynamicNode(35, 0) at 0x2b294225250>, <DynamicNode(35, 1) at 0x2b29413f430>, <DynamicNode(36, 1) at 0x2b29413f880>, <DynamicNode(37, 0) at 0x2b294259670>, <DynamicNode(37, 1) at 0x2b29413f3a0>, <DynamicNode(47, 0) at 0x2b2942593a0>, <DynamicNode(47, 1) at 0x2b29413fbb0>, <DynamicNode(52, 0) at 0x2b29414df40>, <DynamicNode(52, 1) at 0x2b29413f8b0>, <DynamicNode(53, 0) at 0x2b29413ae80>, <DynamicNode(53, 1) at 0x2b29413f6a0>, <DynamicNode(54, 0) at 0x2b286716070>, <DynamicNode(54, 1) at 0x2b2867165e0>, <DynamicNode(55, 0) at 0x2b294259430>, <DynamicNode(55, 1) at 0x2b2942594f0>, <DynamicNode(56, 0) at 0x2b294259f70>, <DynamicNode(56, 1) at 0x2b294259e50>, <DynamicNode(57, 0) at 0x2b294259fd0>, <DynamicNode(57, 1) at 0x2b294259ee0>, <DynamicNode(60, 0) at 0x2b2867165b0>, <DynamicNode(60, 1) at 0x2b29414dac0>, <DynamicNode(61, 0) at 0

In [9]:
print('You need *CPD(s)* for:')
for vS in variableSet:
    lastVariableList.append(vS)
    indexList = list(filter(lambda x: variableList[x] == vS, range(len(variableList))))
    specifiedVAREVIs = []
    for index in indexList:
        specifiedVAREVIs.append(str("(")+str(evidenceList[index]).split("(")[1].split(")")[0]+str(")"))
    lastEvidenceList.append(specifiedVAREVIs)
    print('P(', vS, '|', specifiedVAREVIs, ')')

You need *CPD(s)* for:
P( (24, 0) | ['(54, 0)'] )
P( (24, 1) | ['(24, 0)', '(54, 1)'] )
P( (29, 0) | ['(60, 0)'] )
P( (29, 1) | ['(29, 0)', '(60, 1)'] )
P( (35, 0) | ['(54, 0)'] )
P( (35, 1) | ['(35, 0)', '(54, 1)'] )
P( (36, 1) | ['(36, 0)'] )
P( (37, 0) | ['(55, 0)'] )
P( (37, 1) | ['(37, 0)', '(55, 1)'] )
P( (47, 0) | ['(60, 0)'] )
P( (47, 1) | ['(47, 0)', '(60, 1)'] )
P( (52, 0) | ['(53, 0)'] )
P( (52, 1) | ['(52, 0)', '(53, 1)'] )
P( (53, 0) | ['(54, 0)'] )
P( (53, 1) | ['(53, 0)', '(54, 1)'] )
P( (54, 0) | ['(36, 0)'] )
P( (54, 1) | ['(36, 1)', '(54, 0)'] )
P( (55, 0) | ['(54, 0)'] )
P( (55, 1) | ['(54, 1)', '(55, 0)'] )
P( (56, 0) | ['(55, 0)'] )
P( (56, 1) | ['(55, 1)', '(56, 0)'] )
P( (57, 0) | ['(56, 0)'] )
P( (57, 1) | ['(56, 1)', '(57, 0)'] )
P( (60, 0) | ['(52, 0)', '(61, 0)'] )
P( (60, 1) | ['(52, 1)', '(60, 0)', '(61, 1)'] )
P( (61, 0) | ['(53, 0)'] )
P( (61, 1) | ['(53, 1)', '(61, 0)'] )
P( (64, 0) | ['(56, 0)'] )
P( (64, 1) | ['(56, 1)', '(64, 0)'] )
P( (65, 0) | ['(57

In [10]:
for vS in range(len(lastVariableList)):
    # tabularCPD 작성 반자동
    print('! Please enter the proper *CPDs*,', 2*(2**len(lastEvidenceList[vS])) ,'for:')
    print('P(', lastVariableList[vS], '|', lastEvidenceList[vS], ')')

    cpdInput = list(map(float, input().split())) # 한줄로 cpd를 입력받기 (예시: 0.1 0.2 0.1 0.5 0.3 0.2 0.1)
    if len(cpdInput) != 2*(2**len(lastEvidenceList[vS])) :
        raise Exception('Error: You *MUST* enter', 2*(2**len(lastEvidenceList[vS])), 'cpd!')

    halfCPDInput = math.floor(len(cpdInput) / 2)
    cpdPacked1 = []
    cpdPacked2 = []
    for num in range(halfCPDInput):
        cpdPacked1.append(cpdInput[num])
    for num2 in range(halfCPDInput, len(cpdInput)):
        cpdPacked2.append(cpdInput[num2])

    nums = []
    for n in range(len(lastEvidenceList[vS])):
        nums.append(2)

    cpd = TabularCPD(lastVariableList[vS],
                    2,[cpdPacked1, cpdPacked2],
                    evidence=lastEvidenceList[vS],
                    evidence_card=nums)

    DBN.add_cpds(cpd)
    tabCPDList.append(cpd)
    print(cpd)





! Please enter the proper *CPDs*, 4 for:
P( (24, 0) | ['(54, 0)'] )



FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
print('Root probability of (36, 0)')
root_cpd = TabularCPD((36, 0), 2, [[1], [0]])
DBN.add_cpds(root_cpd)
print(root_cpd, '\n')

pprint.pprint(tabCPDList)
# Check the model
DBN.initialize_initial_state()   ##이걸 안하면 value error 뜸.. 'No CPD associated with aim0030_0'
DBN.check_model()

In [ ]:
print(DBN)
map = {0: 'YES', 1: 'NO'}
dbn_inf = DBNInference(DBN)